## 第7章 モデルの作り方（基本）

### 7.1 いちばんやさしいマス埋め問題

In [1]:
import pulp

# 数理モデル作成
model = pulp.LpProblem()

# 各変数を作成.cat=pulp.LpBinaryでバイナリ変数として作成
Var1 = pulp.LpVariable('Var1', cat=pulp.LpBinary)
Var2 = pulp.LpVariable('Var2', cat=pulp.LpBinary)
Var3 = pulp.LpVariable('Var3', cat=pulp.LpBinary)

# 1*Var1 + 2*Var2 + 3*Var3 == 2 と言う制約条件をモデルに追加
condition = (1*Var1 + 2*Var2 + 3*Var3 == 2)  # 制約条件
model += condition  # 制約条件をモデルに追加

# Var1 + Var2 + Var3 == 1 と言う制約条件をモデルに追加
model += (Var1 + Var2 + Var3 == 1)  # 直接書いても良い

# 数理モデルを解く
model.solve()

# pulp.valueで、最適化された変数を参照
print('Var1', pulp.value(Var1))
print('Var2', pulp.value(Var2))
print('Var3', pulp.value(Var3))
print('Number', pulp.value(1*Var1 + 2*Var2 + 3*Var3))

Var1 0.0
Var2 1.0
Var3 0.0
Number 2.0


#### リストを使って書き換える

In [2]:
import pulp

# 数理モデルを作成
model = pulp.LpProblem()

# 3つの変数をバイナリ変数で作成
Var = [pulp.LpVariable(f'Var{i}', cat=pulp.LpBinary)
       for i in range(3)]

# マスに入る数字の合計が２である制約条件を追加
model += (pulp.lpDot([1, 2, 3], Var) == 2)

# マスに入る数字が1つである制約条件を追加
model += (pulp.lpSum(Var) == 1)

# 数理モデルを解く
model.solve()

# 結果を参照
for v in Var:
    print(v.name, pulp.value(v))
print('Number', pulp.value(pulp.lpDot([1, 2, 3], Var)))

Var0 0.0
Var1 1.0
Var2 0.0
Number 2.0


#### pandasを使った数理モデル

In [3]:
import pulp, pandas

# 数理モデルを作成
model = pulp.LpProblem()

# データフレームに変数と定数を追加
df = pandas.DataFrame()
df['Number'] = [1, 2, 3]
df['Var'] = [pulp.LpVariable(f'Var{i}', cat=pulp.LpBinary)
             for i in range(3)]

# 数理モデルに制約条件を追加
model += (pulp.lpDot(df.Number, df.Var) == 2)
model += (pulp.lpSum(df.Var) == 1)

# 数理モデルを解く
model.solve()

# 結果を表示
df['Val'] = df.Var.apply(pulp.value)
print(df)
print('Number', df[df.Val == 1].Number.iloc[0])

   Number   Var  Val
0       1  Var0  0.0
1       2  Var1  1.0
2       3  Var2  0.0
Number 2


### 7.2 例題:輸送最適化問題

In [4]:
import numpy as np, pandas as pd
from itertools import product
from pulp import LpVariable, lpSum, value
from ortoolpy import model_min, addvars, addvals
np.random.seed(1)
nw, nf = 3, 4
pr = list(product(range(nw), range(nf)))
供給 = np.random.randint(30, 50, nw)
需要 = np.random.randint(20, 40, nf)
輸送費 = np.random.randint(10, 20, (nw,nf))

#### pandasを使わない数理モデル

In [5]:
m1 = model_min()
v1 = {(i, j): LpVariable('v%d_%d' % (i,j), lowBound=0)
     for i, j in pr}
m1 += lpSum(輸送費[i][j] * v1[i, j] for i, j in pr)
for i in range(nw):
    m1 += lpSum(v1[i, j] for j in range(nf)) <= 供給[i]
for j in range(nf):
    m1 += lpSum(v1[i, j] for i in range(nw)) >= 需要[j]
m1.solve()
{k:value(x) for k,x in v1.items() if value(x) > 0}

{(0, 0): 28.0,
 (0, 1): 7.0,
 (1, 2): 31.0,
 (1, 3): 5.0,
 (2, 1): 22.0,
 (2, 3): 20.0}

#### pandasを使った数理モデル

In [6]:
df = pd.DataFrame([(i, j) for i, j in pr],
                  columns=['倉庫', '工場'])
df['輸送費'] = 輸送費.flatten()
df[:3]  # 最初の3行表示

,倉庫,工場,輸送費
0,0,0,10
1,0,1,10
2,0,2,11


In [7]:
m2 = model_min()
addvars(df)
m2 += lpSum(df.輸送費 * df.Var)
for k, v in df.groupby('倉庫'):
    m2 += lpSum(v.Var) <= 供給[k]
for k, v in df.groupby('工場'):
    m2 += lpSum(v.Var) >= 需要[k]
m2.solve()
addvals(df)
df[df.Val > 0]

,倉庫,工場,輸送費,Var,Val
0,0,0,10,v000001,28.0
1,0,1,10,v000002,7.0
6,1,2,12,v000007,31.0
7,1,3,14,v000008,5.0
9,2,1,12,v000010,22.0
11,2,3,12,v000012,20.0


### 7.4 生産最適化を解く

In [8]:
import pandas as pd
from pulp import lpSum, value
from ortoolpy import model_max, addvars, addvals
df0 = pd.read_csv('data/prod_cost.csv', index_col=0)
df0

,原料1,原料2,原料3,利益
製品1,1,4,3,5.0
製品2,2,4,1,4.0
在庫,40,80,50,NaN


In [9]:
df, inv = df0.iloc[:-1, :].copy(), df0.iloc[-1, :-1]
addvars(df)  # 生産量を表す変数
df

,原料1,原料2,原料3,利益,Var
製品1,1,4,3,5.0,v000013
製品2,2,4,1,4.0,v000014


In [10]:
inv

原料1    40.0
原料2    80.0
原料3    50.0
Name: 在庫, dtype: float64

In [11]:
m = model_max()  # 数理モデル
m += lpSum(df.利益 * df.Var)  # 総利益を表す目的関数
for item in df.columns[:-2]: # 製品ごとの処理
    # 制約条件：原料の使用量が在庫以下
    m += lpSum(df[item] * df.Var) <= inv[item]
m.solve()  # ソルバで解を求める
value(m.objective)  # 目的関数の値

95.0

In [12]:
addvals(df)  # 変数の値を表に追加
df

,原料1,原料2,原料3,利益,Var,Val
製品1,1,4,3,5.0,v000013,15.0
製品2,2,4,1,4.0,v000014,5.0


### 7.5 ロジスティクス・ネットワーク設計問題

In [13]:
製品 = list('AB')
需要地 = list('PQ')
工場 = list('XY')
レーン = (2, 2)

In [14]:
import numpy as np, pandas as pd
tbdi = pd.DataFrame(((j, k) for j in 需要地 for k in 工場),
                    columns=['需要地', '工場'])
tbdi['輸送費'] = [1,2,3,1]
tbdi

,需要地,工場,輸送費
0,P,X,1
1,P,Y,2
2,Q,X,3
3,Q,Y,1


In [15]:
tbde = pd.DataFrame(((j, i) for j in 需要地 for i in 製品),
                    columns=['需要地', '製品'])
tbde['需要'] = [10, 10, 20, 20]
tbde

,需要地,製品,需要
0,P,A,10
1,P,B,10
2,Q,A,20
3,Q,B,20


In [16]:
tbfa = pd.DataFrame(((k, l, i, 0, np.inf) for k, nl in
    zip(工場, レーン) for l in range(nl) for i in 製品),
    columns=['工場', 'レーン', '製品', '下限', '上限'])
tbfa['生産費'] = [1, np.nan, np.nan, 1, 3,  np.nan, 5, 3]
tbfa.dropna(inplace=True)
tbfa.loc[4, '上限'] = 10
tbfa

,工場,レーン,製品,下限,上限,生産費
0,X,0,A,0,inf,1.0
3,X,1,B,0,inf,1.0
4,Y,0,A,0,10.000000,3.0
6,Y,1,A,0,inf,5.0
7,Y,1,B,0,inf,3.0


In [17]:
from ortoolpy import logistics_network
_, tbdi2, _ = logistics_network(tbde, tbdi, tbfa)

In [18]:
tbfa[tbfa.ValY > 0]

,工場,レーン,製品,下限,上限,生産費,VarY,ValY
0,X,0,A,0,inf,1.0,v000023,20.0
3,X,1,B,0,inf,1.0,v000024,10.0
4,Y,0,A,0,10.000000,3.0,v000025,10.0
7,Y,1,B,0,inf,3.0,v000027,20.0


In [19]:
tbdi2[tbdi2.ValX > 0]

,需要地,工場,輸送費,製品,VarX,ValX
0,P,X,1,A,v000015,10.0
1,P,X,1,B,v000016,10.0
2,Q,X,3,A,v000017,10.0
6,Q,Y,1,A,v000021,10.0
7,Q,Y,1,B,v000022,20.0


### 7.6 ナンプレを解く

In [20]:
import re, pandas as pd
from itertools import product
from pulp import lpSum, value
from ortoolpy import addbinvars, addvals, model_min

s = ('. . 6 |. . . |. . 1 '
     '. 7 . |. 6 . |. 5 . '
     '8 . . |1 . 3 |2 . . '
     '------+------+------'
     '. . 5 |. 4 . |8 . . '
     '. 4 . |7 . 2 |. 9 . '
     '. . 8 |. 1 . |7 . . '
     '------+------+------'
     '. . 1 |2 . 5 |. . 3 '
     '. 6 . |. 7 . |. 8 . '
     '2 . . |. . . |4 . . ')
data = re.sub(r'[^\d.]','',s)  # 数字とドット以外を削除
r = range(9)
df = pd.DataFrame([(i,j,(i//3)*3+j//3,k+1,c==str(k+1))
    for (i,j),c in zip(product(r,r),data) for k in r],
    columns=['行', '列', '_3x3', '数', '固'])
addbinvars(df)
m = model_min()
for cl in [['行', '列'], ['行', '数'], ['列', '数'],
           ['_3x3', '数']]:
    for _,v in df.groupby(cl):
        m += lpSum(v.Var) == 1
for _,r in df[df.固 == True].iterrows():
    m += r.Var == 1
m.solve()  # ソルバーで求解

1

In [21]:
addvals(df)
print(df[df.Val > 0.5].数.values.reshape(9, 9))

[[5 3 6 8 2 7 9 4 1]
 [1 7 2 9 6 4 3 5 8]
 [8 9 4 1 5 3 2 6 7]
 [7 1 5 3 4 9 8 2 6]
 [6 4 3 7 8 2 1 9 5]
 [9 2 8 5 1 6 7 3 4]
 [4 8 1 2 9 5 6 7 3]
 [3 6 9 4 7 1 5 8 2]
 [2 5 7 6 3 8 4 1 9]]


### 7.7 その他の最適化モデルのテクニック

In [22]:
import numpy as np
from pulp import LpProblem, LpMaximize, lpSum, value
from ortoolpy import (addvar, addvars, addbinvar, addbinvars,
                      addlines, addlines_conv)
m = LpProblem()  # 数理モデル
var = addvars(10)  # 変数リスト

#### 隣接制約

In [23]:
for x in [0, 2, 4]:
    m = LpProblem()
    y = addvar(lowBound=None)
    m += y  # 目的関数（yの最小化）
    m += y >= 2 - x
    m += y >= -2 + x
    m.solve()
    print(f'x, y = {x}, {value(y)}')

x, y = 0, 2.0
x, y = 2, 0.0
x, y = 4, 2.0


In [24]:
M = 4 # 十分大きな数
for x in [0, 2, 4]:
    m = LpProblem()
    y = addvar(lowBound=None)
    z = addbinvar()  # 0-1変数
    m += y  # 目的関数（yの最小化）
    m += y >= 2 - x - M * z
    m += y >= -2 + x - M * (1 - z)
    m.solve()
    print(f'x, y = {x}, {value(y)}')

x, y = 0, -2.0
x, y = 2, 0.0
x, y = 4, -2.0


#### 区分線形近似（非凸）

In [25]:
M = 8
for x in [0, 3, 8, 11, 15]:
    m = LpProblem(sense=LpMaximize)
    y = addvar()
    m += y  # 目的関数
    addlines(m, [(0,5), (3,2), (8,8), (11,5), (15,11)], x, y)
    m.solve()
    print(x, value(y))

0 5.0
3 2.0
8 8.0
11 5.0
15 11.0


#### 区分線形近似（凸）

In [26]:
for x in [0, 3, 6, 9, 13]:
    m = LpProblem(sense=LpMaximize)
    y = addvar()
    m += y  # 目的関数
    addlines_conv(m, [(0,3), (3,8), (6,9), (9,8), (13,3)], x, y,
                  upper=False)
    m.solve()
    print(x, value(y))

0 3.0
3 8.0
6 9.0
9 8.0
13 3.0


#### if A then Bの制約条件

ケース1：$if ~ y \le 10 ~ then ~ y \ge 2 x$

In [27]:
M = 10
for x in [0, 5, 10]:
    m = LpProblem()
    y = addvar()
    z = addbinvar()  # 0-1変数
    m += y  # 目的関数
    m += y >= 10 - M * z  # not A
    m += y <= 10 + M * (1 - z)  # A
    m += y >= 2 * x - M * (1 - z)  # B
    m.solve()
    print(x, value(y))

0 0.0
5 10.0
10 10.0


ケース2：$if ~ x \le 5 ~ then ~ y \le 2 x$

In [28]:
M = 10
for x in [0, 4, 5, 10]:
    m = LpProblem(sense=LpMaximize)
    y = addvar()
    z = addbinvar()  # 0-1変数
    m += y  # 目的関数
    m += x >= 5 - M * z  # not A
    m += x <= 5 + M * (1 - z)  # A
    m += y <= 2 * x + M * (1 - z)  # B
    m.solve()
    print(x, value(y))

0 0.0
4 8.0
5 20.0
10 30.0


ケース3：$if ~ x == 1 ~ then ~ y \le 2$

In [29]:
M = 8  # 十分大きい数とする
for x in [0, 1]:
    m = LpProblem(sense=LpMaximize)
    y = addvar(upBound=10)
    m += y  # 目的関数
    m += y <= 2 + M * (1 - x)
    m.solve()
    print(x, value(y))

0 10.0
1 2.0


ケース4：$if ~ x == 1 ~ then ~ y = 2$

In [30]:
M = 8  # 十分大きい数とする
for x in [0, 1]:
    m = LpProblem(sense=LpMaximize)
    y = addvar(upBound=10)
    m += y  # 目的関数
    m += y <= 2 + M * (1 - x)
    m += y >= 2 - M * (1 - x)
    m.solve()
    print(x, value(y))

0 10.0
1 2.0
